In [12]:
import scipy.io
import itertools
import numpy as np


def acc_measure(idx, Y):
    best_acc = 0
    for idx_order in itertools.permutations([1, 2, 3, 4]):

        for ind, label in enumerate(idx_order):
            Y[(ind)*100:(ind+1)*100] = label

        acc = (Y == idx).sum() / Y.shape[0]
        if acc > best_acc:
            best_acc = acc

    return best_acc

def cluster(bow, K, num_iters = 1000, epsilon = 1e-12):
    max_initializations = 50 #number of random initializations
    best_acc = 0 #initialize variable to store best accuracy
    last_acc = 0 #initialize variable to store accuracy of last initialization
    best_idx = None #variable to store best cluster assignments
    n_c = K #number of clusters
    n_d = len(bow) #number of documents
    n_w = bow.shape[1] #number of words

    #for each initialization
    for init in range(max_initializations):
        pi_c = np.random.rand(n_c) #prior probabilities for clusters
        pi_c /= np.sum(pi_c) #normalize

        #prior probabilities of a particular word being w_j
        mu_jc = np.random.rand(n_w, n_c) 
        mu_jc = mu_jc/np.sum(mu_jc, axis=0) #normalize for each cluster

        #keep track of previous values to check for convergence
        prev_pi_c = np.copy(pi_c)
        prev_mu_jc=np.copy(mu_jc)

        log_mu_jc = np.log(mu_jc+epsilon) #log of probabilities
        log_pi_c = np.log(pi_c+epsilon) #log of priors

        #EM Algorithm
        for it in range(num_iters):
            #E-step
            log_gamma_ic = bow @ log_mu_jc + log_pi_c #calculate log-likelihood
            log_gamma_ic -= np.max(log_gamma_ic, axis=1, keepdims=True) #scale log values
            gamma_ic = np.exp(log_gamma_ic) #convert bavk to probabilities
            gamma_ic /= np.sum(gamma_ic,axis=1,keepdims=True) #normalize
                
            #M-step
            pi_c = np.sum(gamma_ic,axis=0)/n_d #update cluster priors
            mu_jc = (bow.T@gamma_ic) #update probabilities of words in clusters
            mu_jc /= np.sum(mu_jc,axis=0,keepdims=True) #normalize
            #update log probabilities
            log_mu_jc = np.log(mu_jc+epsilon)
            log_pi_c = np.log(pi_c+epsilon)

            #check for convergence
            delta_pi_c = np.linalg.norm(pi_c - prev_pi_c)
            delta_mu_jc = np.linalg.norm(mu_jc - prev_mu_jc, axis=0)
            if delta_pi_c < epsilon and np.all(delta_mu_jc < epsilon):
                break
            #update previous values for next iteration
            prev_pi_c = np.copy(pi_c)
            prev_mu_jc = np.copy(mu_jc)
            
        #assign documents to clusters   
        idx = np.argmax(gamma_ic,axis=1)+1
        #check accuracy of cluster assignments
        acc = acc_measure(idx,Y)
        #update best assignments if current accuracy is better
        if acc>best_acc:
            best_acc = acc
            best_idx = idx
        #stop randomly initializing if accuracy stops changing
        if abs(acc-last_acc)<=epsilon:
            break

        last_acc = acc #update last accuracy
        
    return best_idx

# Evaluation
mat = scipy.io.loadmat('data.mat')
mat = mat['X']
X = mat[:, :-1]
Y = mat[:, -1]
idx = cluster(X, 4)
acc = acc_measure(idx, Y)
print('accuracy %.4f' % (acc))
print('idx: cluster function assignments')
print(idx)
print('Y: true cluster assignments')
print(Y)

accuracy 0.8900
idx: cluster function assignments
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 2 3 3 3 1 2 1 1 3
 3 3 3 1 2 3 1 3 3 2 1 3 3 4 2 3 1 3 3 3 4 4 1 3 1 1 3 3 3 3 3 4 2 1 3 3 3
 3 1 1 3 3 4 3 3 3 4 3 1 3 3 3 1 1 3 1 1 2 1 3 3 3 3 3 3 3 3 2 3 2 4 3 3 3
 3 2 3 3 2 2 4 4 3 3 3 2 3 4 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
Y: true cluster assignments
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 